# Vacancy Formation Energy of Copper

Task: to find the vacancy formation energy in bulk copper

The vacancy formation energy is the energy required to break the bonds between an atom inside the crystal and its nearest neighbor atoms and removing that atom to where it has no interaction with the remaining system[1]. We are calculating the energy of the bulk system using Morse Potential with parameters from [2].

reference: 

[1]https://icme.hpc.msstate.edu/mediawiki/index.php/LAMMPS_Vacancy_Formation_Energy

[2]Oluwajobi, A. & Chen, X. Int. J. Autom. Comput. (2011) 8: 326. https://doi.org/10.1007/s11633-011-0588-y

In [1]:
using JuLIP, JuLIP.Potentials , Optim

In [2]:
# Morse potential
D = 0.3429 # well depth
alpha = 0.13588 # width of potential
r_e = 2.866 # atomic distance at equilibrium
nshell = 2
mp = (@analytic r -> D * ( exp(-2*alpha*(r-r_e)) - 2*exp(-alpha*(r-r_e)) )) * SplineCutoff(0.1, nshell*r_e+0.1)

JuLIP.Potentials.AnalyticFunction{##1#4,##2#5,##3#6}(#1, #2, #3) * JuLIP.Potentials.SplineCutoff(0.1, 5.832)

In [3]:
# reference energy
# energy per unit volume of a homogeneous copper crystal
at = bulk(:Cu)
set_calculator!(at, mp)
Eref = energy(at)

-2.0293100517187743

In [4]:
# cluster with vacancy
at = bulk(:Cu, cubic = true) * 4
set_calculator!(at, mp)
deleteat!(at, length(at) ÷ 2)
# energy before relaxing
E0 = energy(at)

-515.444753136619

In [5]:
# geometry optimization
set_constraint!(at, FixedCell(at))
result = minimise!(at)
E1 = result.minimum

Results of Optimization Algorithm
 * Algorithm: Conjugate Gradient
 * Starting Point: [0.0,0.0, ...]
 * Minimizer: [-3.9729167250894077,-0.5631795008253148, ...]
 * Minimum: -6.243141e+03
 * Iterations: 881
 * Convergence: true
   * |x - x'| ≤ 0.0e+00: false 
     |x - x'| = 4.80e-06 
   * |f(x) - f(x')| ≤ 1.0e-32 |f(x)|: false
     |f(x) - f(x')| = -2.38e-13 |f(x)|
   * |g(x)| ≤ 1.0e-05: true 
     |g(x)| = 8.91e-06 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 1774
 * Gradient Calls: 882


-6243.1410177526595

In [8]:
# defect formation energy
println("Vacancy Formation Energy without relaxing: ", round(E0 - length(at)*Eref, 4))
println("Vacancy Formation Energy with relaxing: ", round(E1 - length(at)*Eref, 4))

Vacancy Formation Energy without relaxing: 2.0293
Vacancy Formation Energy with relaxing: -5725.667
